In [1]:
import dask.dataframe as dd
import pandas as pd
import numpy as np

In [2]:
oscars = pd.read_csv('data/oscars/the_oscar_award.csv')

In [3]:
oscars.head(10)

,year_film,year_ceremony,ceremony,category,name,film,winner
0,1927,1928,1,ACTOR,Richard Barthelmess,The Noose,False
1,1927,1928,1,ACTOR,Emil Jannings,The Last Command,True
2,1927,1928,1,ACTRESS,Louise Dresser,A Ship Comes In,False
3,1927,1928,1,ACTRESS,Janet Gaynor,7th Heaven,True
4,1927,1928,1,ACTRESS,Gloria Swanson,Sadie Thompson,False
5,1927,1928,1,ART DIRECTION,Rochus Gliese,Sunrise,False
6,1927,1928,1,ART DIRECTION,William Cameron Menzies,The Dove;,True
7,1927,1928,1,ART DIRECTION,Harry Oliver,7th Heaven,False
8,1927,1928,1,CINEMATOGRAPHY,George Barnes,The Devil Dancer;,False
9,1927,1928,1,CINEMATOGRAPHY,Charles Rosher,Sunrise,True


In [4]:
oscars_wo_actors = oscars[~(oscars['category'].str.contains('ACTOR') | oscars['category'].str.contains('ACTRESS'))]

# Category Clean up

In [6]:
oscars_wo_actors['category'].unique()

array(['ART DIRECTION', 'CINEMATOGRAPHY', 'DIRECTING (Comedy Picture)',
       'DIRECTING (Dramatic Picture)', 'ENGINEERING EFFECTS',
       'OUTSTANDING PICTURE', 'UNIQUE AND ARTISTIC PICTURE',
       'WRITING (Adaptation)', 'WRITING (Original Story)',
       'WRITING (Title Writing)', 'SPECIAL AWARD', 'DIRECTING', 'WRITING',
       'OUTSTANDING PRODUCTION', 'SOUND RECORDING',
       'SHORT SUBJECT (Cartoon)', 'SHORT SUBJECT (Comedy)',
       'SHORT SUBJECT (Novelty)', 'ASSISTANT DIRECTOR', 'FILM EDITING',
       'MUSIC (Scoring)', 'MUSIC (Song)', 'DANCE DIRECTION',
       'WRITING (Screenplay)', 'SHORT SUBJECT (Color)',
       'SHORT SUBJECT (One-reel)', 'SHORT SUBJECT (Two-reel)',
       'IRVING G. THALBERG MEMORIAL AWARD', 'MUSIC (Original Score)',
       'CINEMATOGRAPHY (Black-and-White)', 'CINEMATOGRAPHY (Color)',
       'SPECIAL EFFECTS', 'ART DIRECTION (Black-and-White)',
       'ART DIRECTION (Color)', 'WRITING (Original Screenplay)',
       'DOCUMENTARY (Short Subject)',
    

In [7]:
oscars_wo_actors['category_clean'] = oscars_wo_actors['category']

/var/folders/hl/m6_sqfln4rz_clzgydgpr1lh0000gn/T/ipykernel_68473/3313081630.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  oscars_wo_actors['category_clean'] = oscars_wo_actors['category']


In [8]:
# Clean up music category

oscars_wo_actors.loc[oscars_wo_actors['category'].str.lower().str.match(r'.*music.*(song).*'), 'category_clean'] = 'MUSIC (Song)'
oscars_wo_actors.loc[oscars_wo_actors['category'].str.lower().str.match(r'.*music.*(score|scoring).*'), 'category_clean'] = 'MUSIC (Score)'

In [9]:
# Clean up sound

oscars_wo_actors.loc[oscars_wo_actors['category'].str.lower().str.match(r'.*sound.*'), 'category_clean'] = 'SOUND'

In [10]:
# Clean up make up

oscars_wo_actors.loc[oscars_wo_actors['category'].str.lower().str.match(r'.*makeup.*'), 'category_clean'] = 'MAKEUP AND HAIRSTYLING'

In [11]:
# Clean up cinematography

oscars_wo_actors.loc[oscars_wo_actors['category'].str.lower().str.match(r'.*cinematography.*'), 'category_clean'] = 'CINEMATOGRAPHY'

In [12]:
# Clean up visual effects

oscars_wo_actors.loc[oscars_wo_actors['category'].str.lower().str.match(r'.*visual.*effect.*'), 'category_clean'] = 'VISUAL EFFECTS'

In [13]:
# Clean up writing 
oscars_wo_actors.loc[oscars_wo_actors['category'].str.lower().str.match(r'.*writing.*'), 'category_clean'] = 'WRITING'

In [14]:
# Clean up directing 
oscars_wo_actors.loc[oscars_wo_actors['category'].str.lower().str.match(r'.*directing.*'), 'category_clean'] = 'DIRECTING'

In [15]:
# Clean up production
oscars_wo_actors.loc[oscars_wo_actors['category'].str.lower().str.match(r'.*(production)|(art direction).*'), 'category_clean'] = 'PRODUCTION DESIGN'

In [16]:
# Clean up costume design 
oscars_wo_actors.loc[oscars_wo_actors['category'].str.lower().str.match(r'.*costume.*'), 'category_clean'] = 'COSTUME DESIGN'

In [17]:
# Clean up documentary
oscars_wo_actors.loc[oscars_wo_actors['category'].str.lower().str.match(r'.*documentary.*'), 'category_clean'] = 'DOCUMENTARY'

In [18]:
# Clean up short subject
oscars_wo_actors.loc[oscars_wo_actors['category'].str.lower().str.match(r'.*short subject.*'), 'category_clean'] = 'SHORT SUBJECT'

In [19]:
# Clean up best picture
oscars_wo_actors.loc[oscars_wo_actors['category'].str.lower().str.match(r'.*(outstanding.*picture)|(best.*picture).*'), 'category_clean'] = 'BEST PICTURE'

In [20]:
oscars_wo_actors['category_clean'].unique()

array(['PRODUCTION DESIGN', 'CINEMATOGRAPHY', 'DIRECTING',
       'ENGINEERING EFFECTS', 'BEST PICTURE',
       'UNIQUE AND ARTISTIC PICTURE', 'WRITING', 'SPECIAL AWARD', 'SOUND',
       'SHORT SUBJECT', 'ASSISTANT DIRECTOR', 'FILM EDITING',
       'MUSIC (Score)', 'MUSIC (Song)', 'DANCE DIRECTION',
       'IRVING G. THALBERG MEMORIAL AWARD', 'SPECIAL EFFECTS',
       'DOCUMENTARY', 'COSTUME DESIGN',
       'SPECIAL FOREIGN LANGUAGE FILM AWARD',
       'HONORARY FOREIGN LANGUAGE FILM AWARD', 'HONORARY AWARD',
       'FOREIGN LANGUAGE FILM', 'JEAN HERSHOLT HUMANITARIAN AWARD',
       'VISUAL EFFECTS', 'SHORT FILM (Animated)',
       'SHORT FILM (Live Action)', 'SPECIAL ACHIEVEMENT AWARD',
       'SHORT FILM (Dramatic Live Action)', 'MAKEUP AND HAIRSTYLING',
       'ANIMATED FEATURE FILM', 'INTERNATIONAL FEATURE FILM',
       'GORDON E. SAWYER AWARD', 'AWARD OF COMMENDATION'], dtype=object)

In [21]:
filtered_categories = [
    'IRVING G. THALBERG MEMORIAL AWARD',
    'HONORARY FOREIGN LANGUAGE FILM AWARD', 
    'HONORARY AWARD',
    'JEAN HERSHOLT HUMANITARIAN AWARD',
    'GORDON E. SAWYER AWARD', 
    'AWARD OF COMMENDATION'
]

In [22]:
oscars_wo_actors = oscars_wo_actors[~oscars_wo_actors['category_clean'].isin(filtered_categories)]

# Name Cleanup

In [24]:
oscars_wo_actors['name'].fillna(value='No Name', inplace=True)
oscars_wo_actors[oscars_wo_actors['name'].str.match(r'.+\s(by|and|&|,|;)\s.+')]

,year_film,year_ceremony,ceremony,category,name,film,winner,category_clean
288,1934,1935,7,MUSIC (Scoring),"RKO Radio Studio Music Department, Max Steiner...",The Gay Divorcee,False,MUSIC (Score)
289,1934,1935,7,MUSIC (Scoring),"RKO Radio Studio Music Department, Max Steiner...",The Lost Patrol,False,MUSIC (Score)
290,1934,1935,7,MUSIC (Scoring),"Columbia Studio Music Department, Louis Silver...",One Night of Love,True,MUSIC (Score)
291,1934,1935,7,MUSIC (Song),Music by Vincent Youmans; Lyrics by Edward Eli...,Flying Down to Rio,False,MUSIC (Song)
292,1934,1935,7,MUSIC (Song),Music by Con Conrad; Lyrics by Herb Magidson,The Gay Divorcee,True,MUSIC (Song)
...,...,...,...,...,...,...,...,...
10880,2023,2024,96,WRITING (Original Screenplay),Screenplay - Justine Triet and Arthur Harari,Anatomy of a Fall,True,WRITING
10881,2023,2024,96,WRITING (Original Screenplay),Written by David Hemingson,The Holdovers,False,WRITING
10882,2023,2024,96,WRITING (Original Screenplay),Written by Bradley Cooper & Josh Singer,Maestro,False,WRITING
10883,2023,2024,96,WRITING (Original Screenplay),Screenplay by Samy Burch; Story by Samy Burch ...,May December,False,WRITING


In [25]:
names_cleaned = oscars_wo_actors['name'].str.extract(r'(?P<name_prefix>.+by )*(?P<name_cleaned>.+)')
oscars_wo_actors = oscars_wo_actors.join(names_cleaned)

In [26]:
oscars_wo_actors['name_cleaned'] = oscars_wo_actors.apply(lambda row: row['name_cleaned'].split(', '), axis=1)
oscars_wo_actors = oscars_wo_actors.explode('name_cleaned')

In [27]:
oscars_wo_actors['name_cleaned'] = oscars_wo_actors.apply(lambda row: row['name_cleaned'].split(' and '), axis=1)
oscars_wo_actors = oscars_wo_actors.explode('name_cleaned')
oscars_wo_actors['name_cleaned'] = oscars_wo_actors.apply(lambda row: row['name_cleaned'].split(' & '), axis=1)
oscars_wo_actors = oscars_wo_actors.explode('name_cleaned')

In [28]:
oscars_wo_actors

,year_film,year_ceremony,ceremony,category,name,film,winner,category_clean,name_prefix,name_cleaned
5,1927,1928,1,ART DIRECTION,Rochus Gliese,Sunrise,False,PRODUCTION DESIGN,NaN,Rochus Gliese
6,1927,1928,1,ART DIRECTION,William Cameron Menzies,The Dove;,True,PRODUCTION DESIGN,NaN,William Cameron Menzies
7,1927,1928,1,ART DIRECTION,Harry Oliver,7th Heaven,False,PRODUCTION DESIGN,NaN,Harry Oliver
8,1927,1928,1,CINEMATOGRAPHY,George Barnes,The Devil Dancer;,False,CINEMATOGRAPHY,NaN,George Barnes
9,1927,1928,1,CINEMATOGRAPHY,Charles Rosher,Sunrise,True,CINEMATOGRAPHY,NaN,Charles Rosher
...,...,...,...,...,...,...,...,...,...,...
10882,2023,2024,96,WRITING (Original Screenplay),Written by Bradley Cooper & Josh Singer,Maestro,False,WRITING,Written by,Bradley Cooper
10882,2023,2024,96,WRITING (Original Screenplay),Written by Bradley Cooper & Josh Singer,Maestro,False,WRITING,Written by,Josh Singer
10883,2023,2024,96,WRITING (Original Screenplay),Screenplay by Samy Burch; Story by Samy Burch ...,May December,False,WRITING,Screenplay by Samy Burch; Story by,Samy Burch
10883,2023,2024,96,WRITING (Original Screenplay),Screenplay by Samy Burch; Story by Samy Burch ...,May December,False,WRITING,Screenplay by Samy Burch; Story by,Alex Mechanik


In [29]:
oscars_wo_actors['first_name'] = oscars_wo_actors['name_cleaned'].str.extract(r'^(\w+)[- ]+.+')

In [30]:
oscars_wo_actors

,year_film,year_ceremony,ceremony,category,name,film,winner,category_clean,name_prefix,name_cleaned,first_name
5,1927,1928,1,ART DIRECTION,Rochus Gliese,Sunrise,False,PRODUCTION DESIGN,NaN,Rochus Gliese,Rochus
6,1927,1928,1,ART DIRECTION,William Cameron Menzies,The Dove;,True,PRODUCTION DESIGN,NaN,William Cameron Menzies,William
7,1927,1928,1,ART DIRECTION,Harry Oliver,7th Heaven,False,PRODUCTION DESIGN,NaN,Harry Oliver,Harry
8,1927,1928,1,CINEMATOGRAPHY,George Barnes,The Devil Dancer;,False,CINEMATOGRAPHY,NaN,George Barnes,George
9,1927,1928,1,CINEMATOGRAPHY,Charles Rosher,Sunrise,True,CINEMATOGRAPHY,NaN,Charles Rosher,Charles
...,...,...,...,...,...,...,...,...,...,...,...
10882,2023,2024,96,WRITING (Original Screenplay),Written by Bradley Cooper & Josh Singer,Maestro,False,WRITING,Written by,Bradley Cooper,Bradley
10882,2023,2024,96,WRITING (Original Screenplay),Written by Bradley Cooper & Josh Singer,Maestro,False,WRITING,Written by,Josh Singer,Josh
10883,2023,2024,96,WRITING (Original Screenplay),Screenplay by Samy Burch; Story by Samy Burch ...,May December,False,WRITING,Screenplay by Samy Burch; Story by,Samy Burch,Samy
10883,2023,2024,96,WRITING (Original Screenplay),Screenplay by Samy Burch; Story by Samy Burch ...,May December,False,WRITING,Screenplay by Samy Burch; Story by,Alex Mechanik,Alex


# Match Gender

In [32]:
baby_names = pd.read_csv('data/oscars/NationalNames.csv')
baby_names['is_female'] = baby_names.apply(lambda row: row['Count'] if row['Gender'] == 'F' else 0, axis=1) 
baby_names['is_male'] = baby_names.apply(lambda row: row['Count'] if row['Gender'] == 'M' else 0, axis=1) 

In [33]:
baby_names[baby_names['Name'] == 'Jonathan']

,Id,Name,Year,Gender,Count,is_female,is_male
1356,1357,Jonathan,1880,M,18,0,18
3306,3307,Jonathan,1881,M,21,0,21
5302,5303,Jonathan,1882,M,27,0,27
7556,7557,Jonathan,1883,M,17,0,17
9708,9709,Jonathan,1884,M,21,0,21
...,...,...,...,...,...,...,...
1745004,1745005,Jonathan,2012,M,9329,0,9329
1768437,1768438,Jonathan,2013,F,12,12,0
1778418,1778419,Jonathan,2013,M,8507,0,8507
1802812,1802813,Jonathan,2014,F,10,10,0


In [34]:
baby_names_agg = baby_names.groupby('Name').agg(female=('is_female', 'sum'), male=('is_male', 'sum')).reset_index('Name')

In [35]:
baby_names_agg[baby_names_agg['Name'] == 'Rick']

,Name,female,male
71920,Rick,135,81251


In [36]:
baby_names_agg['likelihood'] = baby_names_agg.apply(lambda row: row['male'] / (row['male'] + row['female']), axis=1) 

In [37]:
def decide_gender(likelihood):
    if likelihood < 0.5:
        return 'F'
    else:
        return 'M'

baby_names_agg['gender'] = baby_names_agg['likelihood'].apply(lambda likelihood: decide_gender(likelihood))

In [38]:
baby_names_agg[baby_names_agg['likelihood'].between(0.1, 0.9)]

,Name,female,male,likelihood,gender
103,Aalijah,107,155,0.591603,M
135,Aamari,67,88,0.567742,M
189,Aaren,282,717,0.717718,M
190,Aareon,10,33,0.767442,M
195,Aarian,40,76,0.655172,M
...,...,...,...,...,...
93747,Zyian,49,35,0.416667,F
93750,Zyien,12,48,0.800000,M
93754,Zyion,131,629,0.827632,M
93838,Zyon,500,1946,0.795585,M


In [39]:
baby_names_agg = baby_names_agg.rename(columns={'Name' : 'first_name'})

In [40]:
oscars_w_genders = oscars_wo_actors.merge(baby_names_agg[['first_name', 'gender']], how='left', on='first_name')

In [41]:
oscars_w_genders[oscars_w_genders['first_name'].isin(['Luc','Marie'])]

,year_film,year_ceremony,ceremony,category,name,film,winner,category_clean,name_prefix,name_cleaned,first_name,gender
3981,1956,1957,29,COSTUME DESIGN (Color),Marie De Matteis,War and Peace,False,COSTUME DESIGN,NaN,Marie De Matteis,Marie,F
11365,2005,2006,78,DOCUMENTARY (Feature),Luc Jacquet and Yves Darondeau,March of the Penguins,True,DOCUMENTARY,NaN,Luc Jacquet,Luc,M
13811,2018,2019,91,SHORT FILM (Live Action),Marianne Farley and Marie-Hélène Panisset,Marguerite,False,SHORT FILM (Live Action),NaN,Marie-Hélène Panisset,Marie,F
14821,2023,2024,96,BEST PICTURE,"Marie-Ange Luciani and David Thion, Producers",Anatomy of a Fall,False,BEST PICTURE,NaN,Marie-Ange Luciani,Marie,F
14919,2023,2024,96,VISUAL EFFECTS,"Charley Henley, Luc-Ewen Martin-Fenouillet, Si...",Napoleon,False,VISUAL EFFECTS,NaN,Luc-Ewen Martin-Fenouillet,Luc,M


# Save Raw Dataset

In [43]:
# Save file
oscars_w_genders.to_csv('data/oscars/oscars_cleansed.csv')

# Group by Year and Ceremony

## Unique Nominees
**Note:** Unique nominees does not change the over all trend

In [46]:
unique_nominees = oscars_w_genders.groupby(by=['year_ceremony', 'name_cleaned']).agg(gender=('gender', 'max')).reset_index().dropna()

In [47]:
unique_nominees

,year_ceremony,name_cleaned,gender
2,1928,Alfred Cohn,M
3,1928,Anthony Coldeway,M
4,1928,Ben Hecht,M
5,1928,Benjamin Glazer,M
6,1928,Charles Rosher,M
...,...,...,...
12601,2024,Trish Adlesic,F
12602,2024,Troy Quane,M
12604,2024,Vincent René-Lortie,M
12605,2024,Wes Anderson,M


In [48]:
unique_nominees.to_csv('data/oscars/unique_nominees.csv')

## Category Metrics

In [50]:
oscars_w_genders[(oscars_w_genders['year_ceremony'] == 2024) & (oscars_w_genders['category_clean'] == 'WRITING')]

,year_film,year_ceremony,ceremony,category,name,film,winner,category_clean,name_prefix,name_cleaned,first_name,gender
14922,2023,2024,96,WRITING (Adapted Screenplay),Written for the screen by Cord Jefferson,American Fiction,True,WRITING,Written for the screen by,Cord Jefferson,Cord,M
14923,2023,2024,96,WRITING (Adapted Screenplay),Written by Greta Gerwig & Noah Baumbach,Barbie,False,WRITING,Written by,Greta Gerwig,Greta,F
14924,2023,2024,96,WRITING (Adapted Screenplay),Written by Greta Gerwig & Noah Baumbach,Barbie,False,WRITING,Written by,Noah Baumbach,Noah,M
14925,2023,2024,96,WRITING (Adapted Screenplay),Written for the screen by Christopher Nolan,Oppenheimer,False,WRITING,Written for the screen by,Christopher Nolan,Christopher,M
14926,2023,2024,96,WRITING (Adapted Screenplay),Screenplay by Tony McNamara,Poor Things,False,WRITING,Screenplay by,Tony McNamara,Tony,M
14927,2023,2024,96,WRITING (Adapted Screenplay),Written by Jonathan Glazer,The Zone of Interest,False,WRITING,Written by,Jonathan Glazer,Jonathan,M
14928,2023,2024,96,WRITING (Original Screenplay),Screenplay - Justine Triet and Arthur Harari,Anatomy of a Fall,True,WRITING,NaN,Screenplay - Justine Triet,Screenplay,NaN
14929,2023,2024,96,WRITING (Original Screenplay),Screenplay - Justine Triet and Arthur Harari,Anatomy of a Fall,True,WRITING,NaN,Arthur Harari,Arthur,M
14930,2023,2024,96,WRITING (Original Screenplay),Written by David Hemingson,The Holdovers,False,WRITING,Written by,David Hemingson,David,M
14931,2023,2024,96,WRITING (Original Screenplay),Written by Bradley Cooper & Josh Singer,Maestro,False,WRITING,Written by,Bradley Cooper,Bradley,M


In [51]:
oscars_by_year_category = (
    oscars_w_genders.groupby(by=['year_ceremony', 'category_clean', 'category'], as_index=False)
    .value_counts(['gender'])
    .pivot(index=['year_ceremony', 'category_clean', 'category'], columns=['gender'], values=['count'])
    .fillna(0)
    .reset_index()
)
oscars_by_year_category

year_ceremony            category_clean                       category  \
gender                                                                          
0               1928              BEST PICTURE            OUTSTANDING PICTURE   
1               1928            CINEMATOGRAPHY                 CINEMATOGRAPHY   
2               1928                 DIRECTING     DIRECTING (Comedy Picture)   
3               1928                 DIRECTING   DIRECTING (Dramatic Picture)   
4               1928       ENGINEERING EFFECTS            ENGINEERING EFFECTS   
...              ...                       ...                            ...   
1740            2024  SHORT FILM (Live Action)       SHORT FILM (Live Action)   
1741            2024                     SOUND                          SOUND   
1742            2024            VISUAL EFFECTS                 VISUAL EFFECTS   
1743            2024                   WRITING   WRITING (Adapted Screenplay)   
1744            2024                   WRITING  WRITING (Original Screenplay)   

       count        
gender     F     M  
0        0.0   1.0  
1        0.0   3.0  
2        0.0   2.0  
3        0.0   3.0  
4        0.0   3.0  
...      ...   ...  
1740     1.0   7.0  
1741     0.0  19.0  
1742     5.0  14.0  
1743     1.0   5.0  
1744     1.0   6.0  

[1745 rows x 5 columns]

In [52]:
oscars_w_genders[(oscars_w_genders['year_ceremony'] == 2024) & (oscars_w_genders['category'] == 'SOUND')]

,year_film,year_ceremony,ceremony,category,name,film,winner,category_clean,name_prefix,name_cleaned,first_name,gender
14882,2023,2024,96,SOUND,"Ian Voigt, Erik Aadahl, Ethan Van der Ryn, Tom...",The Creator,False,SOUND,NaN,Ian Voigt,Ian,M
14883,2023,2024,96,SOUND,"Ian Voigt, Erik Aadahl, Ethan Van der Ryn, Tom...",The Creator,False,SOUND,NaN,Erik Aadahl,Erik,M
14884,2023,2024,96,SOUND,"Ian Voigt, Erik Aadahl, Ethan Van der Ryn, Tom...",The Creator,False,SOUND,NaN,Ethan Van der Ryn,Ethan,M
14885,2023,2024,96,SOUND,"Ian Voigt, Erik Aadahl, Ethan Van der Ryn, Tom...",The Creator,False,SOUND,NaN,Tom Ozanich,Tom,M
14886,2023,2024,96,SOUND,"Ian Voigt, Erik Aadahl, Ethan Van der Ryn, Tom...",The Creator,False,SOUND,NaN,Dean Zupancic,Dean,M
14887,2023,2024,96,SOUND,"Steven A. Morrow, Richard King, Jason Ruder, T...",Maestro,False,SOUND,NaN,Steven A. Morrow,Steven,M
14888,2023,2024,96,SOUND,"Steven A. Morrow, Richard King, Jason Ruder, T...",Maestro,False,SOUND,NaN,Richard King,Richard,M
14889,2023,2024,96,SOUND,"Steven A. Morrow, Richard King, Jason Ruder, T...",Maestro,False,SOUND,NaN,Jason Ruder,Jason,M
14890,2023,2024,96,SOUND,"Steven A. Morrow, Richard King, Jason Ruder, T...",Maestro,False,SOUND,NaN,Tom Ozanich,Tom,M
14891,2023,2024,96,SOUND,"Steven A. Morrow, Richard King, Jason Ruder, T...",Maestro,False,SOUND,NaN,Dean Zupancic,Dean,M


In [53]:
# drop the extra level, and rename the count columns
oscars_by_year_category.columns = ['_'.join(str(s).strip() for s in col if s) for col in oscars_by_year_category.columns]
oscars_by_year_category

,year_ceremony,category_clean,category,count_F,count_M
0,1928,BEST PICTURE,OUTSTANDING PICTURE,0.0,1.0
1,1928,CINEMATOGRAPHY,CINEMATOGRAPHY,0.0,3.0
2,1928,DIRECTING,DIRECTING (Comedy Picture),0.0,2.0
3,1928,DIRECTING,DIRECTING (Dramatic Picture),0.0,3.0
4,1928,ENGINEERING EFFECTS,ENGINEERING EFFECTS,0.0,3.0
...,...,...,...,...,...
1740,2024,SHORT FILM (Live Action),SHORT FILM (Live Action),1.0,7.0
1741,2024,SOUND,SOUND,0.0,19.0
1742,2024,VISUAL EFFECTS,VISUAL EFFECTS,5.0,14.0
1743,2024,WRITING,WRITING (Adapted Screenplay),1.0,5.0


In [54]:
oscars_by_year_category['change_ratio'] = (
    (oscars_by_year_category['count_M'] - oscars_by_year_category['count_F']) / (oscars_by_year_category['count_M'] + oscars_by_year_category['count_F'])
)

In [55]:
oscars_by_year_category

,year_ceremony,category_clean,category,count_F,count_M,change_ratio
0,1928,BEST PICTURE,OUTSTANDING PICTURE,0.0,1.0,1.000000
1,1928,CINEMATOGRAPHY,CINEMATOGRAPHY,0.0,3.0,1.000000
2,1928,DIRECTING,DIRECTING (Comedy Picture),0.0,2.0,1.000000
3,1928,DIRECTING,DIRECTING (Dramatic Picture),0.0,3.0,1.000000
4,1928,ENGINEERING EFFECTS,ENGINEERING EFFECTS,0.0,3.0,1.000000
...,...,...,...,...,...,...
1740,2024,SHORT FILM (Live Action),SHORT FILM (Live Action),1.0,7.0,0.750000
1741,2024,SOUND,SOUND,0.0,19.0,1.000000
1742,2024,VISUAL EFFECTS,VISUAL EFFECTS,5.0,14.0,0.473684
1743,2024,WRITING,WRITING (Adapted Screenplay),1.0,5.0,0.666667


### Save Yearly Change

In [56]:
oscars_yearly_change = oscars_by_year_category.groupby(by=['year_ceremony']).agg(avg_change=('change_ratio', 'mean'))

In [57]:
oscars_yearly_change.to_csv('data/oscars/oscars_yearly_change.csv')

### Save Yearly Change By Category

In [145]:
oscars_yearly_change_by_category = oscars_by_year_category.groupby(by=['year_ceremony', 'category_clean'], as_index=False).agg(avg_change=('change_ratio', 'mean'))

In [147]:
oscars_yearly_change_by_category.sort_values(by='avg_change', ascending=True)

,year_ceremony,category_clean,avg_change
1261,2019,COSTUME DESIGN,-1.0
1042,2006,COSTUME DESIGN,-1.0
922,1999,COSTUME DESIGN,-1.0
1298,2021,INTERNATIONAL FEATURE FILM,-1.0
1302,2021,PRODUCTION DESIGN,-1.0
...,...,...,...
348,1961,CINEMATOGRAPHY,1.0
826,1993,MAKEUP AND HAIRSTYLING,1.0
827,1993,MUSIC (Score),1.0
829,1993,PRODUCTION DESIGN,1.0


In [149]:
oscars_yearly_change_by_category.to_csv('data/oscars/oscars_yearly_change_by_category.csv')

In [153]:
oscars_yearly_change_by_category[oscars_yearly_change_by_category['year_ceremony'] == 1930]

,year_ceremony,category_clean,avg_change
12,1930,CINEMATOGRAPHY,1.0
13,1930,DIRECTING,1.0
14,1930,PRODUCTION DESIGN,1.0
15,1930,SOUND,1.0
16,1930,WRITING,-1.0


## Yearly Metrics

In [59]:
oscars_yearly = oscars_w_genders.groupby(by=['year_ceremony', 'category']).agg(nominees=('name_cleaned', 'count')).reset_index()

In [60]:
oscars_yearly

,year_ceremony,category,nominees
0,1928,ART DIRECTION,3
1,1928,CINEMATOGRAPHY,3
2,1928,DIRECTING (Comedy Picture),2
3,1928,DIRECTING (Dramatic Picture),3
4,1928,ENGINEERING EFFECTS,3
...,...,...,...
1815,2024,SHORT FILM (Live Action),10
1816,2024,SOUND,20
1817,2024,VISUAL EFFECTS,20
1818,2024,WRITING (Adapted Screenplay),6


In [61]:
oscars_yearly = (
    oscars_yearly
    .groupby(by=['year_ceremony'])
    .agg(num_categories=('category', 'count'), med_nominees=('nominees', 'median'), avg_nominees=('nominees', 'mean'))
).reset_index()

In [62]:
oscars_yearly

,year_ceremony,num_categories,med_nominees,avg_nominees
0,1928,11,3.0,2.818182
1,1929,5,5.0,5.400000
2,1930,6,6.0,7.333333
3,1931,7,5.0,6.142857
4,1932,11,4.0,4.363636
...,...,...,...,...
91,2020,20,7.0,9.850000
92,2021,19,9.0,11.210526
93,2022,19,8.0,11.368421
94,2023,19,8.0,11.473684


In [63]:
oscars_yearly.to_csv('data/oscars/oscars_by_year.csv')

In [137]:
oscars_w_genders[(oscars_w_genders['year_ceremony'].isin(range(1980,1990))) & (oscars_w_genders['category'] == 'DIRECTING')]

,year_film,year_ceremony,ceremony,category,name,film,winner,category_clean,name_prefix,name_cleaned,first_name,gender
7408,1979,1980,52,DIRECTING,Bob Fosse,All That Jazz,False,DIRECTING,NaN,Bob Fosse,Bob,M
7409,1979,1980,52,DIRECTING,Francis Coppola,Apocalypse Now,False,DIRECTING,NaN,Francis Coppola,Francis,M
7410,1979,1980,52,DIRECTING,Peter Yates,Breaking Away,False,DIRECTING,NaN,Peter Yates,Peter,M
7411,1979,1980,52,DIRECTING,Robert Benton,Kramer vs. Kramer,True,DIRECTING,NaN,Robert Benton,Robert,M
7412,1979,1980,52,DIRECTING,Edouard Molinaro,La Cage aux Folles,False,DIRECTING,NaN,Edouard Molinaro,Edouard,M
7589,1980,1981,53,DIRECTING,David Lynch,The Elephant Man,False,DIRECTING,NaN,David Lynch,David,M
7590,1980,1981,53,DIRECTING,Robert Redford,Ordinary People,True,DIRECTING,NaN,Robert Redford,Robert,M
7591,1980,1981,53,DIRECTING,Martin Scorsese,Raging Bull,False,DIRECTING,NaN,Martin Scorsese,Martin,M
7592,1980,1981,53,DIRECTING,Richard Rush,The Stunt Man,False,DIRECTING,NaN,Richard Rush,Richard,M
7593,1980,1981,53,DIRECTING,Roman Polanski,Tess,False,DIRECTING,NaN,Roman Polanski,Roman,M
